In [1]:
import pandas as pd
import numpy as np
import re
import datetime

In [188]:
data = pd.read_json(r'C:\Users\caio-\Documents\Projetos\scrapy_cel\data\data.json',dtype_backend='numpy_nullable')
data.head()

,title,price
0,Celular Samsung Galaxy A10 Tela 6.2'' Dual 32g...,746
1,Celular Samsung J5 16gb Dual Sim Excelente Env...,499
2,Samsung Galaxy A01 Dual Sim 32 Gb Azul 2 Gb Ram,388
3,Motorola Moto G8 Plus Xt2019 Dual 64gb Tela 6....,448
4,Blackberry Bold 9780 256 Mb Black 512 Mb Ram,78


In [189]:
data.dtypes

title    string[python]
price    string[python]
dtype: object

In [190]:
data['price'] = data['price'].str.replace('.','')
data['price'] = data['price'].astype('Int64')

In [191]:
data['brand'] = np.select(
    [
        data['title'].str.contains('Samsung', na=False),
        data['title'].str.contains('Moto|Motorola', na=False),
        data['title'].str.contains('Lenovo', na=False),
        data['title'].str.contains('Blackberry', na=False),
        data['title'].str.contains('LG|Lg|lg', na=False),
        data['title'].str.contains('IPHONE|iPhone|IPhone', na=False)
    ],
    [
        'Samsung',
        'Motorola',
        'Lenovo',
        'Blackberry',
        'LG',
        'iPhone'
    ],
    default='Marca desconhecida'
)

In [192]:
patterns = {
    'Samsung': r'Galaxy \w+',
    'Motorola': r'Moto \w+',
    'Lenovo': r'Lenovo \w+',
    'Blackberry': r'Blackberry\d+',
    'LG': r'(G\d+ \w+)|(K\d+ \w+)',
    'iPhone': r'(iPhone \d+)|(iPhone \w+)'
}

def extract_model(title):
    for brand, pattern in patterns.items():
        match = re.search(pattern, title)
        if match:
            return match.group(0)
    return 'Modelo desconhecido'

data['model'] = data['title'].apply(extract_model)

In [193]:
data['title'] = data['title'].str.upper()

In [194]:
patterns = {
    'a': r'(\d+GB)|(\d+ GB+)'
}

def extract_model(title):
    for brand, pattern in patterns.items():
        match = re.search(pattern, title)
        if match:
            return match.group(0)
    return 'Modelo desconhecido'

data['GB'] = data['title'].apply(extract_model)

In [195]:
data['ultima atualização'] = datetime.datetime.today().date()

In [196]:
data[data['brand']=='iPhone']

,title,price,brand,model,GB,ultima atualização
10,APPLE IPHONE XR 64 GB PRETO - 1 ANO DE GARANTI...,1499,iPhone,iPhone XR,64 GB,2024-06-25
27,IPHONE 7 32 GB PRETO-FOSCO A1778 COMPLETO,599,iPhone,iPhone 7,32 GB,2024-06-25
51,IPHONE 6S 64GB CINZA ESPACIAL CELULAR BOM,489,iPhone,iPhone 6,64GB,2024-06-25
55,APPLE IPHONE 11 64 GB PRETO - 1 ANO DE GARANTI...,1999,iPhone,iPhone 11,64 GB,2024-06-25
67,CELULAR IPHONE 6S 32GB DOURADO USADO MUITO BOM,489,iPhone,iPhone 6,32GB,2024-06-25
...,...,...,...,...,...,...
539,IPHONE 12 128GB ORIGINAL 10X SEM JUROS + BRI...,2589,iPhone,iPhone 12,128GB,2024-06-25
540,IPHONE 8 PLUS 64GB PRATA,1141,iPhone,iPhone 8,64GB,2024-06-25
541,VITRINE - APPLE IPHONE 12 128GB + BRINDES,2469,iPhone,iPhone 12,128GB,2024-06-25
543,APPLE IPHONE 11 (128 GB) - VERDE (RECONDICIONADO),2520,iPhone,iPhone 11,128 GB,2024-06-25


In [197]:
data.to_csv("data.csv",index=False)